Reading JSON data of an existing coupled system

In [1]:
import os
import json


input_path = os.path.join(
    os.path.dirname(os.path.abspath('')),
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

data

{'problem_data': {'start_time': 0.0,
  'end_time': 1.0,
  'echo_level': 0,
  'print_colors': True,
  'parallel_type': 'OpenMP'},
 'solver_settings': {'type': 'coupled_solvers.gauss_seidel_strong',
  'num_coupling_iterations': 20,
  'echo_level': 3,
  'data_transfer_operators': {'direct_transfer': {'type': 'kratos_mapping',
    'mapper_settings': {'mapper_type': 'nearest_neighbor', 'echo_level': 3}}},
  'predictors': [],
  'convergence_accelerators': [{'type': 'aitken',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle'}],
  'convergence_criteria': [{'type': 'relative_norm_previous_residual',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle',
    'abs_tolerance': 1e-07,
    'rel_tolerance': 1e-05}],
  'coupling_sequence': [{'name': 'low_fid_fluid',
    'output_data_list': [],
    'input_data_list': []},
   {'name': 'spring_structure',
    'input_data_list': [{'data': 'lift_force',
      'from_solver': 'low_fid_fluid',
      'from_solver_data': 'lift_force',
      

Connecting to the knowledge base

In [2]:
from coupled_modelling import KnowledgeBase

kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

kb

Importing data of the existing coupled system

In [3]:
mixed_fid_models = kb.import_coupled_kratos('mixed_fid_models', data)

mixed_fid_models

Creating a new coupled system

In [5]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

Checking properties of the imported coupled system

In [6]:
mixed_fid_models.get_properties()

{'label': 'mixed_fid_models',
 'problem_data': 'instance_120',
 'solver_settings': 'instance_121'}

Checking problem data of the imported coupled system

In [7]:
problem_data_old = kb.get_instance(mixed_fid_models.properties['problem_data'])
problem_data_old.get_properties()

{'echo_level': 0,
 'end_time': 1.0,
 'parallel_type': 'OpenMP',
 'print_colors': True,
 'start_time': 0.0}

All properties of the existing problem data instance can be reused, except of echo_level. Copying problem data from the existing coupled system with a new echo_level value

In [ ]:
problem_data_new = problem_data_old.make_copy(
    Onera_FSI,
    {'echo_level': 2})

problem_data_new

Getting solver settings of the imported coupled system

In [ ]:
solver_settings_old = kb.get_instance(mixed_fid_models.properties['solver_settings'])
solver_settings_old.get_properties()

The copying method does not copy nested instances. Thus, we can copy the solver settings from the existing coupled system with the old *num_coupling_iterations* and *type*, but with a new *echo_level*

In [ ]:
solver_settings_new = solver_settings_old.make_copy(
    Onera_FSI,
    {'echo_level': 4})

solver_settings_new

Getting existing convergence accelerators

In [ ]:
convergence_accelerators_old = kb.get_instance(solver_settings_old.properties['convergence_accelerators'][0])
convergence_accelerators_old.get_properties()

Only *type* can be reused, thus, we create new convergence accelerators for the new coupled system from scratch, passing the class, the parent instance and new property values

In [ ]:
convergence_accelerators_new = kb.create_instance(
    'convergence_accelerators',
    solver_settings_new,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators_new

Checking existing convergence criteria

In [ ]:
convergence_criteria_old = kb.get_instance(solver_settings_old.properties['convergence_criteria'][0])
convergence_criteria_old.get_properties()

Copying convergence criteria, passing new *data_name*, *solver* and *type*

In [ ]:
convergence_criteria_new = convergence_criteria_old.make_copy(solver_settings_new, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

convergence_criteria_new

Checking existing coupling sequences

In [ ]:
coupling_sequence_old_0 = kb.get_instance(solver_settings_old.properties['coupling_sequence'][0])
coupling_sequence_old_0.get_properties()

In [ ]:
coupling_sequence_old_1 = kb.get_instance(solver_settings_old.properties['coupling_sequence'][1])
coupling_sequence_old_1.get_properties()

Checking existing input data list

In [ ]:
input_data_list_old = kb.get_instance(coupling_sequence_old_1.properties['input_data_list'][0])
input_data_list_old.get_properties()

Checking existing output data list

In [ ]:
output_data_list_old = kb.get_instance(coupling_sequence_old_1.properties['output_data_list'][0])
output_data_list_old.get_properties()

We create a new coupling sequence by creating its elements one by one

In [ ]:
coupling_sequence_new_0 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'CFD'})

coupling_sequence_new_0

Creating another coupling sequence

In [ ]:
coupling_sequence_new_1 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'SM'})

coupling_sequence_new_1

Then we create new nested instances for the second item in the new coupling sequence. Copying a new input data list

In [ ]:
input_data_list_new = input_data_list_old.make_copy(coupling_sequence_new_1, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

input_data_list_new

Creating a new output data list

In [ ]:
output_data_list_new = kb.create_instance('output_data_list', coupling_sequence_new_1, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

output_data_list_new

Checking existing data transfer operators

In [ ]:
data_transfer_operators_old = kb.get_instance(solver_settings_old.properties['data_transfer_operators'])
data_transfer_operators_old.get_properties()

Checking existing mapper settings

In [ ]:
direct_transfer = kb.get_instance(data_transfer_operators_old.properties['mapper_settings'])
direct_transfer.get_properties()

Creating a mapping operation instance

In [ ]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings_new, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

mapping_operation

Creating new mapper settings

In [ ]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

mapper_settings

Checking the first existing solver

In [ ]:
solver_old_0 = kb.get_instance(solver_settings_old.properties['solvers'][0])
solver_old_0.get_properties()

Creating an instance of the CFD solver by copying the first solver

In [ ]:
CFD = solver_old_0.make_copy(solver_settings_new, {
    'label': 'CFD'})

CFD

Checking existing data

In [ ]:
data_old_0_0 = kb.get_instance(solver_old_0.properties['data'][0])
data_old_0_0.get_properties()

In [ ]:
data_old_0_1 = kb.get_instance(solver_old_0.properties['data'][1])
data_old_0_1.get_properties()

Creating displacements data for the CFD solver instance

In [ ]:
displacements_0 = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

displacements_0

Creating lift force data instance for the CFD solver

In [ ]:
lift_force_0 = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

lift_force_0

Checking existing IO settings

In [ ]:
io_settings_old = kb.get_instance(solver_old_0.properties['io_settings'])
io_settings_old.get_properties()

Copying io settings

In [ ]:
io_settings_new = io_settings_old.make_copy(CFD, {
    'connect_to': 'run_SU2'})

io_settings_new

Checking existing solver wrapper settings

In [ ]:
solver_wrapper_settings_old_0 = kb.get_instance(solver_old_0.properties['solver_wrapper_settings'])
solver_wrapper_settings_old_0.get_properties()

Copying solver wrapper settings

In [ ]:
solver_wrapper_settings_new_0 = solver_wrapper_settings_old_0.make_copy(CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

solver_wrapper_settings_new_0

Checking another existing solver

In [ ]:
solver_old_1 = kb.get_instance(solver_settings_old.properties['solvers'][1])
solver_old_1.get_properties()

Creating an instance for the SM solver

In [ ]:
SM = kb.create_instance('solvers', solver_settings_new, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

SM

Checking existing data of the second solver

In [ ]:
data_old_1_0 = kb.get_instance(solver_old_1.properties['data'][0])
data_old_1_0.get_properties()

In [ ]:
data_old_1_1 = kb.get_instance(solver_old_1.properties['data'][1])
data_old_1_1.get_properties()

Coping data for the second solver from the first one

In [ ]:
displacements_1 = displacements_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

displacements_1

Copying another data from the first solver

In [ ]:
lift_force_1 = lift_force_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

lift_force_1

Checking wrapper settings of the second existing solver

In [ ]:
solver_wrapper_settings_old_1 = kb.get_instance(solver_old_1.properties['solver_wrapper_settings'])
solver_wrapper_settings_old_1.get_properties()

Creating solver wrapper settings for the second new solver

In [ ]:
solver_wrapper_settings_new_1 = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

solver_wrapper_settings_new_1

Exporting created coupled system in JSON

In [ ]:
export = Onera_FSI.export_kratos()

export

Saving exported data into a file

In [ ]:
export_path = os.path.join(
    os.path.abspath(''),
    'export_onera_fsi.json')

with open(export_path, 'w') as file:
    json.dump(export, file, indent=2)

Inferring new classes from the created coupled system

In [ ]:
Onera_FSI.infer_classes()

Updating the knowledge base

In [ ]:
kb.save()

Getting a local copy of the knowledge base

In [ ]:
onto_path = os.path.join(
    os.path.abspath(''),
    'demo_client.owl')

kb.save_locally(onto_path)